# 1. Import Libraries and Set Directory
---

Import Libraries

In [4]:
is_drive = 0

In [2]:
if is_drive==1 :
    from google.colab import drive
    drive.mount('/content/drive')

In [7]:
# directory management
import os
from glob import glob
# regular imports
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

# imports for NN
import tensorflow as tf
from tensorflow import keras
import time

# 2. Load Dataset
---

In [5]:
# Spectogram Path
if is_drive==1 :   
    TRAIN_DIR = "/content/drive/MyDrive/spectogram-dataset/Train/"
    TEST_DIR = "/content/drive/MyDrive/spectogram-dataset/Test/"
    AE_SPECTROGRAMS_SAVE_DIR1 = "/content/drive/MyDrive/autoencoder-generations/Generated-Test/"
    E_SPECTROGRAMS_SAVE_DIR1 = "/content/drive/MyDrive/autoencoder-generations/Compressed-Test/"
    AE_SPECTROGRAMS_SAVE_DIR2 = "/content/drive/MyDrive/autoencoder-generations/Generated-Train/"
    E_SPECTROGRAMS_SAVE_DIR2 = "/content/drive/MyDrive/autoencoder-generations/Compressed-Train/"
else :
    TRAIN_DIR = "../Project/spectogram-dataset/Train/"
    TEST_DIR = "../Project/spectogram-dataset/Test/"
    AE_SPECTROGRAMS_SAVE_DIR1 = "../Project/autoencoder-generations/Generated-Test/"
    E_SPECTROGRAMS_SAVE_DIR1 = "../Project/autoencoder-generations/Compressed-Test/"
    AE_SPECTROGRAMS_SAVE_DIR2 = "../Project/autoencoder-generations/Generated-Train/"
    E_SPECTROGRAMS_SAVE_DIR2 = "../Project/autoencoder-generations/Compressed-Train/"

In [1]:
def load_fsdd(the_path):
    x_train = []
    for root, _, file_names in os.walk(the_path):
        for file_name in file_names:
            file_path = os.path.join(root, file_name)
            spectrogram = np.load(file_path) # (n_bins, n_frames)
            x_train.append(spectrogram)
    x_train = np.array(x_train)        # -> (n_samples, n_bins, n_frames)
    x_train = x_train[..., np.newaxis] # -> (1200, 800, 32, 1)
    
    return x_train

In [8]:
x_train_raw = load_fsdd(TRAIN_DIR)
x_test_raw  = load_fsdd(TEST_DIR)

AE_test = load_fsdd(AE_SPECTROGRAMS_SAVE_DIR1)
E_test = load_fsdd(E_SPECTROGRAMS_SAVE_DIR1)

AE_train = load_fsdd(AE_SPECTROGRAMS_SAVE_DIR2)
E_train = load_fsdd(E_SPECTROGRAMS_SAVE_DIR2)

AE_test = AE_test[:, :, :, :, 0]
AE_train = AE_train[:, :, :, :, 0]

In [11]:
print("x_train_raw" , x_train_raw.shape)
print("x_test_raw" , x_test_raw.shape)
print("\n")
print("AE_test" , AE_test.shape)
print("E_test" , E_test.shape)
print("\n")
print("AE_train" , AE_train.shape)
print("E_train" , E_train.shape)

x_train_raw (1200, 160, 84, 1)
x_test_raw (300, 160, 84, 1)


AE_test (300, 160, 84, 1)
E_test (300, 16, 1)


AE_train (1200, 160, 84, 1)
E_train (1200, 16, 1)


In [12]:
num_classes = 10
iterations = 0
i = 0
y_train = []
while iterations!=len(x_train_raw):
    iterations +=1
    y_train.append(i)
    i += 1
    if i == num_classes :
        i = 0
y_train = np.array(y_train)
#---------------------------------------------------------------#
iterations = 0
i = 0
y_test =[]
while iterations!=len(x_test_raw):
    iterations +=1
    y_test.append(i)
    i += 1
    if i == num_classes :
        i = 0
y_test = np.array(y_test)

y_train=keras.utils.to_categorical(y_train,num_classes)
y_test =keras.utils.to_categorical(y_test,num_classes)

In [13]:
print(y_train[0])
print(y_train[5])
print(y_test[9])

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


# 3. Create Classifer

In [14]:
import keras
from keras import layers
from keras import Model

In [22]:


model4 = keras.Sequential()

# Convolutional layer 1
model4.add(layers.Conv2D(4, (3, 3), activation='tanh', input_shape=(16, 1, 1), padding='valid'))

# Average pooling layer 1
model4.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

# Convolutional layer 2
model4.add(layers.Conv2D(32, (5, 5), activation='tanh', padding='valid'))

# Average pooling layer 2
model4.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

# Flatten layer
model4.add(layers.Flatten())

# Fully connected layer 1
model4.add(layers.Dense(120, activation='tanh'))

# Fully connected layer 2
model4.add(layers.Dense(84, activation='tanh'))

# Output layer
model4.add(layers.Dense(10, activation='softmax'))

model4.summary() 

ValueError: One of the dimensions in the output is <= 0 due to downsampling in conv2d_8. Consider increasing the input size. Received input shape [None, 1, 1, 4] which would produce output shape with a zero or negative value in a dimension.